In [1]:
from noworkflow.now.persistence.models import Evaluation, Activation
from noworkflow.now.models.dependency_querier import DependencyQuerier
from noworkflow.now.models.dependency_querier.node_context import NodeContext
from noworkflow.now.models.dependency_querier.querier_options import QuerierOptions
from noworkflow.now.tagging.var_tagging import *
from noworkflow.now.persistence.models import Trial
from copy import copy

import numpy as np
from pprint import pprint

In [2]:
import numpy as np

# Helper functions
def get_activation(activation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Activation((trial_id, activation_id))

def get_evaluation(evaluation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Evaluation((trial_id, evaluation_id))

def get_act_evals(activation_id):
    activation = get_activation(activation_id)
    return activation.evaluations    

def print_act_forward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependent_dependencies):
        pprint.pprint(i.to_dict())
        
def print_act_backward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependency_dependencies):
        pprint.pprint(i.to_dict())

def print_act_evals(activation_id=1):
    activation = get_activation(activation_id)

    for item in activation.evaluations:
        pprint.pprint(item.to_dict())

### Test case

Here starts my (only) test case. The flow goes through cells defining module, mae and rmse, variable definitions, variable manipulation and _var_final_ being our target variable, being dependent of its previous cells.


In [3]:
def module(num): 
    if num >= 0:
        return int(num)
    else:
        return int(-1*num)

def mae(x, y):
    return np.mean(np.abs((x - y)))

def rmse(x, y):
    return np.sqrt(np.mean((x - y)**2))

def stress_func(m, n):
    temp = m * n
    x = 10
    y = 100
    
    temp = np.random.rand()*temp
    
    final = np.max([int(x+y), int(temp)])
    
    return final

In [4]:
var_a = 10

In [24]:
var_b = 0

In [23]:
x = 3665
y = 30

In [7]:
rme_err = mae(var_a, var_b)

In [8]:
var_c = stress_func(x, y)

In [20]:
var_d = var_c * 423
var_c = var_d / 3

In [21]:
var_e = mae(var_c, var_b)
x = var_e + 10

In [25]:
#k = module(rmse_err)
var_final = now_variable('var_final', rmse(var_c, x))

Evaluation(id=330, checkpoint=51.373983231, code_component_id=766, activation_id=324, repr=75423063.00000001)


In [12]:
evals = get_evaluation(135)

In [19]:
evals

[evaluation(43937cee-226d-4a2a-a1ab-b0577d162b9a, 150, 3.7762716350000005, 498, 128, '110.0', 148).]

### New functions test

In [32]:
from noworkflow.now.persistence.models.base import proxy_gen
from noworkflow.now.persistence import relational 
from noworkflow.now.persistence.models import Evaluation as EvaluationM, CodeComponent

def global_history(self):
    # create an enumerated dictionary        
    dep_dict = {i[0] : i[1] for i in reversed(list(enumerate(self.dep_list)))}

    return dep_dict

evals = list(proxy_gen(relational
                       .session
                       .query(EvaluationM.m)
                       .join(CodeComponent.m, ((EvaluationM.m.trial_id == CodeComponent.m.trial_id) & (EvaluationM.m.code_component_id == CodeComponent.m.id)))
                       .filter((CodeComponent.m.name == 'var_final') & (CodeComponent.m.trial_id == __noworkflow__.trial_id))))

In [33]:
evals

[evaluation(43937cee-226d-4a2a-a1ab-b0577d162b9a, 150, 3.7762716350000005, 498, 128, '110.0', 148).,
 evaluation(43937cee-226d-4a2a-a1ab-b0577d162b9a, 315, 41.8789043, 728, 294, '110.0', 313).,
 evaluation(43937cee-226d-4a2a-a1ab-b0577d162b9a, 345, 51.374186746, 759, 324, '75423063.00000001', 343).]

In [28]:
nbOptions = NotebookQuerierOptions(level = True)
querier = DependencyQuerier(options=nbOptions)
nodes_to_visit, visited, found = querier.navigate_dependencies(evals)

In [29]:
len(visited)

495

In [30]:
a = nbOptions.global_history()

In [31]:
a

{90: ("'var_final'", "'var_final'"),
 89: ('2', '2'),
 88: ('mae(var_c, var_b)', '535041.3333333334'),
 87: ('var_e', '535041.3333333334'),
 86: ('var_e', '535041.3333333334'),
 85: ('10', '10'),
 84: ('x', '535051.3333333334'),
 83: ('rmse(var_c, x)', '110.0'),
 82: ("now_variable('var_final', rmse(var_c, x))", '110.0'),
 81: ('var_final', '110.0'),
 80: ("'var_final'", "'var_final'"),
 79: ('2', '2'),
 78: ('100', '100'),
 77: ('var_b', '-100'),
 76: ('mae(var_c, var_b)', '75426828.00000001'),
 75: ('var_e', '75426828.00000001'),
 74: ('var_e', '75426828.00000001'),
 73: ('10', '10'),
 72: ('x', '75426838.00000001'),
 71: ('rmse(var_c, x)', '110.0'),
 70: ("now_variable('var_final', rmse(var_c, x))", '110.0'),
 69: ('var_final', '110.0'),
 68: ("'var_final'", "'var_final'"),
 67: ('2', '2'),
 66: ('10', '10'),
 65: ('x', '10'),
 64: ('100', '100'),
 63: ('y', '100'),
 62: ('int(x+y)', '110'),
 61: ('temp', '1500'),
 60: ('temp', '1187.3611192664782'),
 59: ('int(temp)', '1187'),
 58: